# Structural Estimation PS4
Meng Yang
## Estimating the Brock and Mirman model by SMM

In [1]:
import numpy as np 
import pandas as pd 
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts 
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [2]:
cd '/Users/kate/Desktop/PS4/data'

/Users/kate/Desktop/PS4/data


In [3]:
data = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
c_t, k_t, w_t, r_t, y_t = np.loadtxt('NewMacroSeries.txt', unpack = 'True', delimiter = ',')
df = pd.DataFrame(data, columns = ['c_t', 'k_t', 'w_t', 'r_t', 'y_t'])
df.head()

,c_t,k_t,w_t,r_t,y_t
0,1.128323e+07,8.040697e+06,1.120211e+07,1.008852,1.931398e+07
1,1.215464e+07,8.030754e+06,1.206726e+07,1.088112,2.080561e+07
2,1.097303e+07,8.650974e+06,1.089414e+07,0.911904,1.878300e+07
3,9.711635e+06,7.809971e+06,9.641815e+06,0.893986,1.662382e+07
4,9.245673e+06,6.912184e+06,9.179203e+06,0.961637,1.582621e+07


Six moments: 
* $m_1$ = mean($c_t$)
* $m_2$ = mean($k_t$)
* $m_3$ = mean($c_t/y_t$)
* $m_4$ = var($y_t$)
* $m_5$ = corr($c_t, c_{t-1}$)
* $m_6$ = corr($c_t, k_t$)

### (a) estimate ($\alpha, \rho, \mu, \sigma$) 

* step 1: generate simulated value for $z_{st} z_t = \rho z_{t-1} + (1-\rho)\mu +\varepsilon_t$
* step 2: generate simulated value for $k_{t+1}=\alpha \beta e^{z_t}k_t^{\alpha}$
* step 3: generate simulated value for $w$ and $r$ $w_t = (1-\alpha)e^{z_t}(k_t)^{\alpha}, r_t = \alpha e^{z_t}(k_t)^{\alpha-1}$
* step 4: generate simulated value for $c$ and $y$ $c_t=w_t+r_tk_t-k_{t+1}, y_t = e^{z_t}(k_t)^{\alpha}$

In [4]:
N = 100
S = 1000
beta = 0.99
np.random.seed(25)
unif_vals = sts.uniform.rvs(0,1, size = (N, S))

In [5]:
# define a function to calculate autocorrelation coefficient
def auto(x):
    
    x = pd.Series(x)
    auto = x.autocorr(lag = 1)
    
    return auto

def corr(x,y):
   
    x = pd.Series(x)
    y = pd.Series(y)
    corr = x.corr(y)
    
    return corr

In [6]:
def data_moments(c_t, k_t, w_t, r_t, y_t):
    
    m1_data = c_t.mean()
    m2_data = k_t.mean()
    m3_data = np.mean(c_t/y_t)
    m4_data = np.var(y_t)
    m5_data = auto(c_t)    
    m6_data = corr(c_t, k_t)
    
    return m1_data, m2_data, m3_data, m4_data, m5_data, m6_data

In [7]:
m1_data, m2_data, m3_data, m4_data, m5_data, m6_data = data_moments(c_t, k_t, w_t, r_t, y_t)
m_data = np.array([m1_data, m2_data, m3_data, m4_data, m5_data, m6_data])
print('m1: mean(c_t) = ', m1_data)
print('m2: mean(k_t) = ', m2_data)
print('m3: mean(c_t/y_t) = ', m3_data)
print('m4: var(y_t) = ', m4_data)
print('m5: corr(c_t, c_{t-1}) = ', m5_data)
print('m6: corr(c_t, k_t) = ', m6_data)

m1: mean(c_t) =  9281790.485669706
m2: mean(k_t) =  6643985.138299068
m3: mean(c_t/y_t) =  0.5842000000000002
m4: var(y_t) =  28377825058899.727
m5: corr(c_t, c_{t-1}) =  0.9405591814596135
m6: corr(c_t, k_t) =  0.9408030537975822


In [8]:
def model_moments(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma):
    
    # step1: generate simulated error term  
    varepsilon = sts.norm.ppf(unif_vals, loc = 0, scale = sigma)
    
    # step2: generate simulated z_st
    z_sim = np.zeros((N,S))
    z_sim[0,:] = rho * mu + (1-rho) * mu + varepsilon[0,:]
    for i in range(1,N):
        z_sim[i: ] = rho * z_sim[i-1,:] + (1-rho)*mu + varepsilon[i:]
        
    # step3: generate simultaed k_st
    k_sim_temp = np.zeros((N + 1, S))  
    k_sim_temp[0, :] = k_t.mean()
    for i in range(1, N + 1):
        k_sim_temp[i, :] = alpha * beta * np.exp(z_sim[i - 1, :]) * k_sim_temp[i - 1, :] ** alpha
    
    k_sim = k_sim_temp[:-1, :]  
    k_lead = k_sim_temp[1:, :]  
    
    # step4: generate simulated value for w_st, r_st
    w_sim = np.zeros((N,S))
    r_sim = np.zeros((N,S))
    
    for i in range(N):
        w_sim[i, :] = (1 - alpha) * np.exp(z_sim[i, :]) * k_sim[i, :] ** alpha
        r_sim[i, :] = alpha * np.exp(z_sim[i, :]) * (k_sim[i, :]) ** (alpha - 1)
    
    # step5: generate simulated value for c_st, y_st
    c_sim = np.zeros((N,S))
    y_sim = np.zeros((N,S))
    for i in range(N):
        c_sim[i, :] = w_sim[i, :] + r_sim[i, :] * k_sim[i, :] - k_lead[i, :]
        y_sim[i, :] = np.exp(z_sim[i, :]) * (k_sim[i, :]) ** alpha
    
    # now that we have simulated all the data, we can calculate simulated model moments
    
    m1_model = c_sim.mean(axis=0)
    m1_model = m1_model.mean()
    
    m2_model = k_sim.mean(axis=0)
    m2_model = m2_model.mean()
    
    m3_model = np.mean((c_sim / y_sim), axis = 0)
    m3_model = np.mean(m3_model)
    
    m4_model = np.var(y_sim, axis=0)
    m4_model = m4_model.mean()
        
    m5 = [auto(c_sim[:,i]) for i in range(S)]  
    m5_model = np.mean(m5)
    
    m6 = [corr(c_sim[:,i], k_sim[:,i]) for i in range(S)]
    m6_model = np.mean(m6)
    
    return m1_model, m2_model, m3_model, m4_model, m5_model, m6_model 

In [9]:
def err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma, simple):
    
    dm1, dm2, dm3, dm4, dm5, dm6 = data_moments(c_t, k_t, w_t, r_t, y_t)
    moms_data = np.array([[dm1], [dm2], [dm3], [dm4], [dm5], [dm6]])
    m1_model, m2_model, m3_model, m4_model, m5_model, m6_model = model_moments(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma)
    m_model = np.array([m1_model, m2_model, m3_model, m4_model, m5_model, m6_model])
    
    if simple:
        err_vec = m_model - m_data
    else:
        err_vec = (m_model - m_data) / m_data
    
    return err_vec

def crit(params, *args):
    
    alpha, rho, mu, sigma = params
    c_t, k_t, w_t, r_t, y_t, unif_vals, W = args
    
    err = err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma, simple = False)
    crit_val = err.T @ W @ err
    
    return crit_val

In [10]:
alpha_0 = k_t/(w_t + k_t)
alpha_0 = alpha_0.mean()
rho_0 = 0.7881732669300365 # from PS3
mu_0 = 9.18135946009693
sigma_0 = 1
params_0 = np.array([alpha_0, rho_0, mu_0, sigma_0])
W_0 = np.eye(6)
smm_args = (c_t, k_t, w_t, r_t, y_t, unif_vals, W_0)
print('params_0 = ', params_0)
bnds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))

results_SMM = opt.minimize(crit, params_0, args = (smm_args), bounds = bnds, method = 'L-BFGS-B', tol = 1e-5)
results_SMM

params_0 =  [0.41931596 0.78817327 9.18135946 1.        ]


      fun: 2.9822133943014513
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00071849,  0.00011751, -0.00228195, -0.00119376])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 145
      nit: 15
   status: 0
  success: True
        x: array([ 0.42400047, -0.8754591 ,  5.0524026 ,  0.0682687 ])

In [11]:
alpha_smm,rho_smm, mu_smm, sigma_smm = results_SMM.x
theta = np.array([alpha_smm, rho_smm, mu_smm, sigma_smm])
crit_val = results_SMM.fun
err_val = err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha_smm, rho_smm, mu_smm, sigma_smm, simple = True)
print('theta = ', theta)
print('Value of criterion function = ', crit_val)
print('Moment difference at optimum = ', err_val)

theta =  [ 0.42400047 -0.8754591   5.0524026   0.0682687 ]
Value of criterion function =  2.9822133943014513
Moment difference at optimum =  [-9.27901398e+06 -6.57555102e+06 -3.96046804e-03 -2.83776825e+13
 -4.26584225e-04  5.37962263e-02]


\begin{equation}
  d(\tilde{x},x|\theta) \equiv
    \begin{bmatrix}
      \frac{\partial e_1(\tilde{x},x|\theta)}{\partial \theta_1} & \frac{\partial e_1(\tilde{x},x|\theta)}{\partial \theta_2} & ... & \frac{\partial e_1(\tilde{x},x|\theta)}{\partial \theta_K} \\
      \frac{\partial e_2(\tilde{x},x|\theta)}{\partial \theta_1} & \frac{\partial e_2(\tilde{x},x|\theta)}{\partial \theta_2} & ... & \frac{\partial e_2(\tilde{x},x|\theta)}{\partial \theta_K} \\
      \vdots & \vdots & \ddots & \vdots \\
      \frac{\partial e_R(\tilde{x},x|\theta)}{\partial \theta_1} & \frac{\partial e_R(\tilde{x},x|\theta)}{\partial \theta_2} & ... & \frac{\partial e_R(x|\theta)}{\partial \theta_K}
    \end{bmatrix}
\end{equation}

\begin{equation}
  \hat{\Sigma}_{SMM} = \frac{1}{S}\left[d(\tilde{x},x|\theta)^T W d(\tilde{x},x|\theta)\right]^{-1}
\end{equation}

In [14]:
def Jac_err(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma, simple):
    
    R = 6 
    K = 4
    Jac = np.zeros((6, 4))
    
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    
    Jac[:, 0] = ((err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha + h_alpha, rho, mu, sigma, simple))- (err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha - h_alpha, rho, mu, sigma, simple))) / (2 * h_alpha).flatten() 
    Jac[:, 1] = ((err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho + h_rho, mu, sigma, simple))- (err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho - h_rho, mu, sigma, simple))) / (2 * h_rho).flatten()
    Jac[:, 2] = ((err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu + h_mu, sigma, simple))- (err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu - h_mu, sigma, simple))) / (2 * h_mu).flatten()
    Jac[:, 3] = ((err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma + h_sigma, simple))- (err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma - h_sigma, simple))) / (2 * h_sigma).flatten() 
        
    return Jac

In [15]:
err = Jac_err(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha_smm, rho_smm, mu_smm, sigma_smm, simple = False)
SigHat = (1 / 1000) * lin.inv(err.T @ W_0 @ err)

print('Std of alpha = ', np.sqrt(SigHat[0,0]))
print('Std of rho = ', np.sqrt(SigHat[1,1]))
print('Std of mu = ', np.sqrt(SigHat[2,2]))
print('Std of sigma = ', np.sqrt(SigHat[3,3]))

Std of alpha =  0.018660629970650804
Std of rho =  158.04869285234219
Std of mu =  49.37984838522423
Std of sigma =  34.98382611758998


### (b) Two step SMM

$$ E(\tilde{x},x|\theta) =
  \begin{bmatrix}
    \frac{m_1(\tilde{x}_1|\theta) - m_1(x)}{m_1(x)} & \frac{m_1(\tilde{x}_2|\theta) - m_1(x)}{m_1(x)} & ... & \frac{m_1(\tilde{x}_S|\theta) - m_1(x)}{m_1(x)} \\
    \frac{m_2(\tilde{x}_1|\theta) - m_2(x)}{m_2(x)} & \frac{m_2(\tilde{x}_2|\theta) - m_2(x)}{m_2(x)} & ... & \frac{m_2(\tilde{x}_S|\theta) - m_2(x)}{m_2(x)} \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{m_R(\tilde{x}_1|\theta) - m_R(x)}{m_R(x)} & \frac{m_R(\tilde{x}_2|\theta) - m_R(x)}{m_R(x)} & ... & \frac{m_R(\tilde{x}_S|\theta) - m_R(x)}{m_R(x)} \\
  \end{bmatrix} $$

$$ \hat{\Omega}_2 = \frac{1}{S}E(\tilde{x},x|\hat{\theta}_{1,SMM})\,E(\tilde{x},x|\hat{\theta}_{1,SMM})^T $$

In [16]:
def get_err_mat(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma, simple = False):
    
    R = 6
    S = 1000
    err_mat = np.zeros((R,S))
    m1_data, m2_data, m3_data, m4_data, m5_data, m6_data = data_moments(c_t, k_t, w_t, r_t, y_t)
    m_data = np.array([m1_data, m2_data, m3_data, m4_data, m5_data, m6_data])
    
    m1_model, m2_model, m3_model, m4_model, m5_model, m6_model = model_moments(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha, rho, mu, sigma)
    m_model = np.array([m1_model, m2_model, m3_model, m4_model, m5_model, m6_model])
    
    if simple:
        err_mat[0,:] = m1_model - m1_data
        err_mat[1,:] = m2_model - m2_data
        err_mat[2,:] = m3_model - m3_data
        err_mat[3,:] = m4_model - m4_data
        err_mat[4,:] = m5_model - m5_data
        err_mat[5,:] = m6_model - m6_data
    else:
        err_mat[0,:] = (m1_model - m1_data)/m1_data
        err_mat[1,:] = (m2_model - m2_data)/m2_data
        err_mat[2,:] = (m3_model - m3_data)/m3_data
        err_mat[3,:] = (m4_model - m4_data)/m4_data
        err_mat[4,:] = (m5_model - m5_data)/m5_data
        err_mat[5,:] = (m6_model - m6_data)/m6_data
    
    return err_mat

In [17]:
err_mat = get_err_mat(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha_smm, rho_smm, mu_smm, sigma_smm, simple = False)
VCV2 = (1/ S) * (err_mat @ err_mat.T)
print(VCV2)
W_2step = lin.inv(VCV2)
print(W_2step)
smm_args_2s = (c_t, k_t, w_t, r_t, y_t, unif_vals, W_2step)
params_2s = ([alpha_smm,rho_smm, mu_smm, sigma_smm])

[[ 9.99401820e-01  9.89403785e-01  6.77727375e-03  9.99695842e-01
   4.53407533e-04 -5.71640725e-02]
 [ 9.89403785e-01  9.79505771e-01  6.70947378e-03  9.89694867e-01
   4.48871636e-04 -5.65922020e-02]
 [ 6.77727375e-03  6.70947378e-03  4.59589312e-05  6.77926761e-03
   3.07470620e-06 -3.87648452e-04]
 [ 9.99695842e-01  9.89694867e-01  6.77926761e-03  9.99989952e-01
   4.53540925e-04 -5.71808901e-02]
 [ 4.53407533e-04  4.48871636e-04  3.07470620e-06  4.53540925e-04
   2.05701438e-07 -2.59341344e-05]
 [-5.71640725e-02 -5.65922020e-02 -3.87648452e-04 -5.71808901e-02
  -2.59341344e-05  3.26968705e-03]]
[[-4.34901464e+14  1.33707365e+14 -9.49972572e+16  3.03848195e+14
   8.39868067e+17 -4.57656115e+15]
 [ 1.34431993e+14 -2.27961533e+13 -2.75019778e+15  3.79291499e+11
  -2.76332318e+17 -5.55486026e+14]
 [-9.59165428e+16 -2.62585344e+15 -8.85221986e+18  2.92362460e+16
   1.14856412e+20 -1.34957141e+18]
 [ 3.17053040e+14 -3.17417535e+12  3.14783648e+16  8.87896346e+13
  -6.71571075e+17  5.446

In [18]:
results_SMM_2s = opt.minimize(crit, params_0, args = (smm_args_2s), bounds = bnds, method = 'L-BFGS-B', tol = 1e-5)
results_SMM_2s

      fun: -1.2956691176738655e+21
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.07644987e+21,  9.93183295e+20, -1.54478838e+18, -1.81469184e+18])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 50
      nit: 9
   status: 0
  success: True
        x: array([ 0.01      , -0.99      ,  5.01650806,  0.02783095])

In [25]:
alpha_smm_2s,rho_smm_2s, mu_smm_2s, sigma_smm_2s = results_SMM_2s.x
theta_2s = np.array([alpha_smm_2s,rho_smm_2s, mu_smm_2s, sigma_smm_2s])
crit_2s = results_SMM_2s.fun
err_val_2s = err_vec2(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha_smm_2s,rho_smm_2s, mu_smm_2s, sigma_smm_2s, simple = False)
print('theta = ', theta_2s)
print('Value of criterion function = ', crit_2s)
print('Moment difference at optimum = ', err_val_2s)
err_2s = Jac_err(unif_vals, c_t, k_t, w_t, r_t, y_t, alpha_smm_2s, rho_smm_2s, mu_smm_2s, sigma_smm_2s, simple = False)
SigHat2 = (1 / S) * lin.inv(err_2s.T @ W_2step @ err_2s)

theta =  [ 0.01       -0.99        5.01650806  0.02783095]
Value of criterion function =  -1.2956691176738655e+21
Moment difference at optimum =  [-0.99998364 -0.98999977  0.6947963  -1.         -2.00125476 -0.84965885]
